In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import diffusionstuff7 as ds
from copy import copy as dup
from scipy.integrate import odeint
from matplotlib import rcParams
from test_ice_model_module import multiple_test_avg_time

ModuleNotFoundError: No module named 'diffusionstuff6'

In [ ]:
#Meta testing parameters
n_tests = 50

In [ ]:
# Graphics parameters
%matplotlib widget
ticklabelsize = 15
fontsize = 15
linewidth = 2
fignum = 0

In [ ]:
#Parameters for the model
nx = 500 # Number of points in simulation box
xmax = 50
x = np.linspace(0, xmax, nx)
boxpoints = len(x)
deltaX = x[1]-x[0]

niter = 1 # Number of iterations to calculate Nliq from Ntot
Nbar = 1.0 # new Nbar from VMD, 260K
Nstar = .9/(2*np.pi)
Nice = np.zeros(boxpoints)+1 

In [ ]:
# Timing Initialize as a pre-equilibrated layer of liquid over ice
func = ds.getNliq_array
args = (Nice,Nstar,Nbar,niter)
multiple_test_avg_time(func, args, n_tests)
Fliq = func(*args)

In [ ]:
# Lay out the system
Ntot = Fliq + Nice
nmid = int(nx/2)
nquart = int(nx/4)
xmid = max(x)/2
xmax = x[nx-1]
Nice_start = Nice[0]

Niceoffset = ds.getNiceoffset(Nbar, Nstar, 1, 0) # Should be close to .75; not used if D=const

In [ ]:
#Test diffusion functions setup
# Just conversions
nmpermonolayer = 0.3
umpersec_over_mlyperus = (nmpermonolayer/1e3*1e6)

# Diffusion coefficient
D = 0.02e-2 # micrometers^2/microsecond

# Time steps
dtmaxtimefactor = 10
dtmaxtimefactor = 50
dtmax = deltaX**2/D
deltaT = dtmax/dtmaxtimefactor

# Deposition rate
nu_kin = 49 # microns/second
deprate = nu_kin/umpersec_over_mlyperus # monolayers per microsecond
deprate_times_deltaT = deprate * deltaT

# Supersaturation
sigma0 = 0.19
sigmastepmax = 0.20 # Must be bigger than sigma0 to get growth
center_reduction = 0.25 # In percent
c_r = center_reduction/100

# Diffusion coefficient scaled for this time-step and space-step
DoverdeltaX2 = D/deltaX**2

In [ ]:
#Test sigmastep function performance
func = ds.getsigmastep
method ='sinusoid'
args = (x,xmax,center_reduction,sigmastepmax, method)
multiple_test_avg_time(func, args, n_tests  )

In [ ]:
# Gets the function sigma(x)
sigmastep_sinusoid = ds.getsigmastep(x,xmax,center_reduction,sigmastepmax,method='sinusoid')
sigmastep_parabolic = ds.getsigmastep(x,xmax,center_reduction,sigmastepmax,method='parabolic')
sigmastepstyle = 'sinusoid'
if sigmastepstyle=='sinusoid':
    sigmastep = sigmastep_sinusoid
elif sigmastepstyle=='parabolic':
    sigmastep = sigmastep_parabolic
else:
    print('bad choice')

# Graph sigma(x)
fignum +=1
plt.figure(fignum)
plt.plot(x-xmid,sigmastep_sinusoid/sigmastepmax, \
         x-xmid, sigmastep_parabolic/sigmastepmax, '--',lw=linewidth)
plt.xlim([-xmid,xmid])
plt.legend(['sinusoidal ', 'parabolic'])
plt.xlabel(r'x ($\mu m$)',fontsize=fontsize)
plt.ylabel(r'$\sigma_I(x) $',fontsize=fontsize)
plt.grid('on')

In [ ]:
# These are run control parameters

# Flag for explicit updating Fliq(Ntot) every step
updatingFliq = True

# Set up a maximum number of iterations or layers
uselayers = True

if uselayers:
    layermax_0D = 4
    layermax_1D = 10
    layermax_2D = 10
else:
    countermax_0D = 100
    countermax_1D = 15000
    countermax_2D = 100000


In [ ]:
# This is the 0-d run
timelist = []
for run in range(n_tests):
    starttime = time.time()
    # Bundle parameters for ODE solver
    params = np.array([Nbar, Nstar, sigmastepmax, sigma0, deprate])

    # Initial conditions for ODE solver
    y0 = [Nbar, 0.0]
    ylast = dup(y0)

    # Time steps
    t0 = 0.0
    ttot = 0.0
    tinterval = [t0, deltaT]

    # Initialize the keeper arrays
    tkeep_0D = [t0]
    ykeep_0D = [y0]
    tlast = dup(t0)
    Nicelast0 = 0.0
    Ntotlast0 = 0.0
    Nicelast_start = 0.0
    Ntotlast_start = 0.0

    # Call the ODE solver
    counter = 0
    layer = 0
    while True:
        
        # Integrate up to next time step
        y = odeint(ds.f0d, ylast, tinterval, args=(params,niter))
        ylast = list(y[1,:])
        tlast += deltaT
        
        # Stuff into keeper arrays
        ykeep_0D.append(ylast)
        tkeep_0D.append(tlast)
        
        # Make some local copies, with possible updates to Fliq
        Fliqlast, Ntotlast = ylast

        if updatingFliq:
            Fliqlast = ds.getNliq(Ntotlast,Nstar,Nbar,niter) # This updates Fliqlast and ylast!
            ykeep_0D[counter][0] = Fliqlast # Saves the updated Fliq for this step
        Nicelast = Ntotlast - Fliqlast
        Nicelast0 = dup(Nicelast)
        Ntotlast0 = dup(Ntotlast)
        
        # Update counters and see whether to break
        counter += 1
        layer = dup(Ntotlast0) 
        if uselayers:
            if layer > layermax_0D-1:
                break
        else:
            if counter > countermax_0D-1:
                break
        
    # Convert results to a numpy array
    ykeep_0Darr = np.array(ykeep_0D, np.float64)
    tkeep_0Darr = np.array(tkeep_0D, np.float64)
    timelist.append(time.time()-starttime)

print("Average time to run for ", n_tests, " runs: ", np.mean(timelist), " seconds")

In [ ]:
# Plot results of zero-d run
fignum +=1
plt.figure(fignum)
rcParams['xtick.labelsize'] = ticklabelsize 
rcParams['ytick.labelsize'] = ticklabelsize
plt.plot(tkeep_0D,ykeep_0Darr[:,0],lw=linewidth)
plt.xlabel(r't ($\mu s$)',fontsize=fontsize)
plt.ylabel(r'$N_{QLL} $',fontsize=fontsize)
plt.grid('on')

# Growth statistics
delta_Ntot_0d = Ntotlast0-Ntotlast_start
growthrate_0d_mlyperus = delta_Ntot_0d/tlast # monolayer/us
growthrate_0d = growthrate_0d_mlyperus*umpersec_over_mlyperus # um/sec
print( "0-D Modeled growth rate, um/s", growthrate_0d)
print( "0-D Modeled growth rate, ml/us", growthrate_0d_mlyperus)
alpha_0d = growthrate_0d/nu_kin/sigmastepmax
print( "0-D Modeled alpha", alpha_0d)
timelist.append(time.time()-starttime)

In [ ]:
#Parameters for 1-D run 
L = xmax/2; print(L) # micrometers
c_r = center_reduction / 100; print(c_r) # dimensionless
nu_kin_ml = deprate; print(nu_kin_ml) # monolayers per microsecond
sigma_I = sigmastepmax; print(sigma_I) # dimensionless
print(D) # D is in micrometers^2/microsecond
M = np.array([.0027, .0025])
B = np.array([2.9, 1.59])
beta = np.array([0.65, 0.65])
xfactor = nu_kin_ml*L**2*c_r**beta*sigma_I/D
NSS = M*xfactor + B
print('Nss predicted')
print('sinusoid:', NSS[0])
print('paraboloic:', NSS[1])

In [ ]:
timelist = []
for run in range(n_tests):
    # Timing
    sec1 = time.time()
    #Nbar, Nstar, niter, sigmastep, sigma0, deprate, DoverdeltaX2, nx = params #original params
    # Bundle parameters for ODE solver
    float_params = np.array([Nbar, Nstar, sigma0, deprate, DoverdeltaX2])
    int_params = np.array([niter,nx])

    # Initial conditions for ODE solver
    y0 = list([Fliq,Ntot])
    ylast = dup(y0)

    # Time steps
    t0 = 0.0
    tlast = dup(t0)
    ttot = 0.0
    tinterval = [t0, deltaT]

    # Initialize the keeper arrays
    tkeep = [t0]
    Nicekeep = [dup(Nice)]
    Ntotkeep = [dup(Ntot)]
    Fliqkeep = [dup(Fliq)]
    fluxderivskeep = []

    # Call the ODE solver
    Nice0_start = Nice[0]
    Nice0 = Nice[0]
    Ntot0_start = Ntot[0]
    Ntot0 = Ntot[0]
    updatingFliq = True
    counter = 0
    lastlayer = 0
    lastdiff = 0
    while True:
        # Integrate up to next time step
        y = odeint(ds.f1d, np.reshape(ylast,2*nx), tinterval, args=(float_params,int_params,sigmastep),rtol=1e-12)
        
        ylast = np.reshape(y[1],(2,nx))
        tlast += deltaT
        counter += 1
            
        # Make some local copies, with possible updates to Fliq
        Fliq, Ntot = ylast
        if updatingFliq:
            Fliq = ds.getNliq_array(Ntot,Nstar,Nbar,niter) # This updates to remove any drift
            ylast[0] = Fliq
        Nice = Ntot - Fliq
        Nice0 = Nice[0]
        Ntot0 = Ntot[0]
        ttot += deltaT
        
        # Update counters and see whether to break
        layer = Ntot0-Ntot0_start
        if (layer-lastlayer) > 0:
            minpoint = min(Nice)
            maxpoint = max(Nice)
            print(counter-1, lastlayer, maxpoint-minpoint, maxpoint-minpoint-lastdiff)
            lastdiff = maxpoint-minpoint
            lastlayer += 1
        if uselayers:
            if layer > layermax_1D-1:
                print('breaking because reached max number of layers')
                break
        else:
            if counter > countermax_1D-1:
                print('breaking because reached max number of iterations')
                break
    timelist.append(time.time()-sec1)
print("Took ", np.average(timelist), " seconds (on average for ", n_tests," runs) to run 1d model")

In [ ]:
# Growth statistics
# minpoint = min(Nice)
minpoint = 0
ncorner = 4
print("Height of Ice", minpoint)
delta_Ntot_entire = Ntot-Ntotkeep[0]
print( "1-D growth, corner", delta_Ntot_entire[ncorner])
print( "1-D growth, center", delta_Ntot_entire[nmid])
growthrate_entire = delta_Ntot_entire/ttot*umpersec_over_mlyperus
print( "1-D Modeled growth rate, corner, um/s", growthrate_entire[ncorner])
print( "1-D Modeled growth rate, corner, ml/us", growthrate_entire[ncorner]/umpersec_over_mlyperus)
print( "1-D Modeled growth rate, center, um/s", growthrate_entire[nmid])
alpha = growthrate_entire/nu_kin/sigmastep
print( "1-D Modeled alpha, corner", alpha[ncorner])
print( "1-D Modeled alpha, center", alpha[nmid])
print( "Difference, %", (alpha[nmid]-alpha[ncorner])/alpha[ncorner]*100)


# Comparisons with 0-D model
print( "0-D Modeled growth rate, um/s", growthrate_0d)
print( "0-D Modeled growth rate, ml/us", growthrate_0d/umpersec_over_mlyperus)
print( "0-D Modeled alpha", alpha_0d)
print( "Growth ratio 1D (corner)/0D", growthrate_entire[ncorner]/growthrate_0d)
print( "Growth ratio 1D (center)/0D", growthrate_entire[nmid]/growthrate_0d)
print( "Contribution of diffusion", (growthrate_entire[nmid]-growthrate_0d)/growthrate_0d*100, '%')

# Comparisons with Libbrecht
sigma0_L = 0.08
A_L = .28
alpha_L = A_L*np.exp(-sigma0_L/sigmastepmax)
print("Libbrecht's predicted growth rate, um/s", nu_kin*alpha_L*sigmastepmax)
print("Libbrecht's predicted alpha", alpha_L)

# Timing statistics
sec2 = time.time()
print("Time taken:", int((sec2-sec1)/60), "min", (sec2-sec1)%60, "secs")

# Plot
fignum +=1
plt.figure(fignum)
plt.plot(x-xmid, Nice-minpoint, 'k', label='ice', lw=linewidth)
plt.plot(x-xmid, Fliq+Nice-minpoint, 'b', label='ice+liquid', lw=linewidth)
plt.xlabel(r'$x (\mu m$)',fontsize=fontsize)
plt.ylabel(r'$ice \ layers$',fontsize=fontsize)
plt.xlim([-xmid, xmid])
rcParams['xtick.labelsize'] = ticklabelsize 
rcParams['ytick.labelsize'] = ticklabelsize
plt.legend()
plt.grid('on')

# Plot
fignum +=1
plt.figure(fignum)
plt.plot(x-xmid, Fliq, 'g', label='liquid', lw=linewidth)
plt.xlabel(r'$x (\mu m$)',fontsize=fontsize)
plt.ylabel(r'$ice \ layers$',fontsize=fontsize)
plt.xlim([-xmid, xmid])
rcParams['xtick.labelsize'] = ticklabelsize 
rcParams['ytick.labelsize'] = ticklabelsize
plt.legend()
plt.grid('on')

# Plot
fignum +=1
plt.figure(fignum)
plt.plot(x-xmid, alpha, 'k', lw=linewidth)
plt.xlabel(r'$x (\mu m$)',fontsize=fontsize)
plt.ylabel(r'$ \alpha $',fontsize=fontsize)
rcParams['xtick.labelsize'] = ticklabelsize 
rcParams['ytick.labelsize'] = ticklabelsize
plt.grid('on')

In [ ]:
#Setting up the 2D system

nx = 500 # Number of points in simulation box
xmax = 50 # range of x
x = np.linspace(0, xmax, nx)

ny = nx
ymax = xmax
y = np.linspace(0, ymax, ny)

deltaX = x[1]-x[0]
deltaY = y[1]-y[0]

Nbar = 1.0 # new Nbar from VMD, 260K
Nstar = .9/(2*np.pi)
# Initialize as a pre-equilibrated layer of liquid over ice

Nice = np.ones((nx,ny))

if noisy_init:
    # Initialize with noise
    noise = np.random.normal(0,noise_std_dev,(nx,ny))
    Nice += noise


Fliq = ds.getNliq_2d_array(Nice,Nstar,Nbar,niter)

# Lay out the system
Ntot = Fliq + Nice
# nmid = int(nx/2)
# nquart = int(nx/4)
# xmid = max(x)/2
# xmax = x[nx-1]
Nice_start = Nice[0]

#Niceoffset = ds.getNiceoffset(Nbar, Nstar, 1, 0) # Should be close to .75; not used if D=const #NOTE: this is unused

In [ ]:
#2d run and test
timelist = []
for run in range(n_tests):
    # Timing
    sec1 = time.time()
    # This is the 2-d run

    # Bundle parameters for ODE solver
    float_params = np.array([Nbar, Nstar, sigma0, deprate, DoverdeltaX2])
    int_params = np.array([niter,nx,ny])

    # Initial conditions for ODE solver
    y0 = np.array([Fliq,Ntot])
    ylast = dup(y0)

    # Initialize the keeper arrays
    tkeep = [t0]
    Nicekeep = [dup(Nice)]
    Ntotkeep = [dup(Ntot)]
    Fliqkeep = [dup(Fliq)]
    fluxderivskeep = []

    # Call the ODE solver
    Nice0_start = Nice[0,0]
    Nice0 = Nice[0,0]
    Ntot0_start = Ntot[0,0]
    Ntot0 = Ntot[0,0]
    updatingFliq = True
    counter = 0
    lastlayer = 0
    lastdiff = 0
    while True:
        # Integrate up to next time step
        y = odeint(ds.f2d, np.reshape(ylast,np.prod(np.shape(ylast))), tinterval, args=(float_params,int_params,sigmastep),rtol=1e-12)
        # Update the state                  #NOTE: prod(shape(ylast)) is like (2*nx*ny) #unused--np.prod(np.shape(ylast))
        ylast = np.reshape(y[1],(2,nx,ny))


        tlast += deltaT
        counter += 1
            
        # Make some local copies, with possible updates to Fliq
        Fliq, Ntot = ylast
        if updatingFliq:
            Fliq = ds.getNliq_2d_array(Ntot,Nstar,Nbar,niter) # This updates to remove any drift
            ylast[0] = Fliq
        Nice = Ntot - Fliq

        Nice0 = Nice[0,0]
        Ntot0 = Ntot[0,0]
        ttot += deltaT

        # Stuff into keeper arrays if requested -- needed for making graphics
        trajflag = True
        if trajflag:
            Nicekeep.append(Nice)
            Ntotkeep.append(Ntot)
            Fliqkeep.append(Fliq)
            tkeep.append(ttot)

        # Update counters and see whether to break
        layer = Ntot0-Ntot0_start
        if (layer-lastlayer) > 0:
            minpoint = np.min(Nice)
            maxpoint = np.max(Nice)
            print(counter-1, lastlayer, maxpoint-minpoint, maxpoint-minpoint-lastdiff)
            lastdiff = maxpoint-minpoint
            lastlayer += 1
            
        # Test whether we're finished
        if uselayers:
            print("appx progress:" , ((layer/layermax_2D)*100, "%"))
            if sigmastepmax > 0:
                if layer > layermax_2D-1:
                    print('breaking because reached max number of layers grown')
                    break
            else:
                if layer < -layermax_2D:
                    print('breaking because reached max number of layers ablated')
                    break
        else:
            if counter > countermax_2D-1:
                print('breaking because reached max number of iterations')
                break
    timelist.append(time.time()-sec1)
print("Took ", np.average(timelist), " seconds (on average for ", n_tests," runs) to run 1d model")